In [1]:
import warnings
import sys
warnings.filterwarnings("ignore", message="Trying algorithm eng36")
sys.stderr = open('/dev/null', 'w') if sys.platform.startswith('linux') or sys.platform.startswith('darwin') else open('nul', 'w')

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.saving import load_model
import numpy as np
from keras.callbacks import ModelCheckpoint

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
data_dir = '/kaggle/input/apparel-dataset'

In [7]:
import os


# Function to print directory tree recursively
def print_directory_tree(directory, indent=''):
    for item in os.listdir(directory):
        full_path = os.path.join(directory, item)
        if os.path.isdir(full_path):
            print(indent + item + '/')
            print_directory_tree(full_path, indent + '    ')

# Print the directory tree
print_directory_tree(data_dir)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/apparel-dataset'

In [ ]:
dataset_apparel = image_dataset_from_directory(
    directory = data_dir,
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=16,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='both',#if subset = both it returns train and validation set respectively
)

In [ ]:
for labels in dataset_apparel:
  print(labels)

In [ ]:
checkpoint_path = '/kaggle/working/model_checkpoint.keras'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='auto',
    save_best_only=True,
    verbose=1)

In [ ]:
model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),
    
    layers.Conv2D(512, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2), 
    
    layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(37, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',#multilabel classification
              metrics=['accuracy','auc'])

In [ ]:
history = model.fit(train_set, epochs=10, validation_data=validation_set, callbacks=[model_checkpoint_callback])

print("------------*************------------")
test_loss, test_acc, test_auc = model.evaluate(validation_set)
print(f"Test accuracy: {test_acc}")